In [1]:
!pip install transformers

In [2]:
!pip install pandas

In [3]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
import numpy as np
from datasets import Features, Value

In [6]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import numpy as np
import torch
from typing import Optional

# Custom Trainer class to ensure contiguous tensors
class CustomTrainer(Trainer):
    def save_model(self, output_dir: Optional[str] = None, _internal_call: bool = False):
        # Make all model parameters contiguous before saving
        for param in self.model.parameters():
            if not param.is_contiguous():
                param.data = param.data.contiguous()

        # Call the parent class's save_model method
        super().save_model(output_dir, _internal_call)

# Load pre-trained MedBERT model and tokenizer
model_name = "Charangan/MedBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set num_labels according to the number of classes in your readmission risk prediction task
num_labels = 3  # For example: 'low', 'medium', 'high'

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Define paths to datasets
dataset_paths = {
    'admissions': '/content/drive/MyDrive/dataset/admissions.csv',
    'caregiver': '/content/drive/MyDrive/dataset/caregiver.csv',
    'chartevents': '/content/drive/MyDrive/dataset/chartevents.csv',
    'd_hcpcs': '/content/drive/MyDrive/dataset/d_hcpcs.csv',
    'd_icd_diagnoses': '/content/drive/MyDrive/dataset/d_icd_diagnoses.csv',
    'd_icd_procedures': '/content/drive/MyDrive/dataset/d_icd_procedures.csv',
    'd_labitems': '/content/drive/MyDrive/dataset/d_labitems.csv',
    'diagnoses_icd': '/content/drive/MyDrive/dataset/diagnoses_icd.csv',
    'drgcodes': '/content/drive/MyDrive/dataset/drgcodes.csv',
    'emar': '/content/drive/MyDrive/dataset/emar.csv',
    'hcpcsevents': '/content/drive/MyDrive/dataset/hcpcsevents.csv',
    'icustays': '/content/drive/MyDrive/dataset/icustays.csv',
    'labevents': '/content/drive/MyDrive/dataset/labevents.csv',
    'microbiologyevents': '/content/drive/MyDrive/dataset/microbiologyevents.csv',
    'outputevents': '/content/drive/MyDrive/dataset/outputevents.csv',
    'patients': '/content/drive/MyDrive/dataset/patients.csv',
    'pharmacy': '/content/drive/MyDrive/dataset/pharmacy.csv',
    'procedures_icd': '/content/drive/MyDrive/dataset/procedures_icd.csv',
    'services': '/content/drive/MyDrive/dataset/services.csv',
    'transfers': '/content/drive/MyDrive/dataset/transfers.csv',
    'final_features':'/content/drive/MyDrive/dataset/finall_features.csv',
    'target_variables_all_fin':'/content/drive/MyDrive/dataset/target_variables_all_fin.csv'
}

def tokenize_and_format(examples):
    # Construct text from available fields
    text = f"{examples['admission_type']} {examples['admission_location']} {examples['discharge_location']}"

    # Tokenize
    tokenized = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=275,
    )

    # Add a dummy label (replace this with your actual label logic)
    tokenized['labels'] = 0

    return tokenized

# Load and prepare dataset
dataset = load_dataset('csv', data_files={'train': dataset_paths['admissions']}, delimiter=',')

# Print dataset info
print("Dataset info:")
print(dataset)
print("\nDataset columns:", dataset['train'].column_names)

# Prepare the dataset
tokenized_dataset = dataset['train'].map(tokenize_and_format, remove_columns=dataset['train'].column_names)

# Print sample
print("\nSample from tokenized dataset:")
print(tokenized_dataset[0])

# Verify dataset structure
print("\nDataset features:")
print(tokenized_dataset.features)

# Split dataset
train_dataset = tokenized_dataset.shuffle(seed=42).select(range(275))
eval_dataset = tokenized_dataset.shuffle(seed=42).select(range(100))

# Define compute_metrics function
def compute_metrics(p):
    metric = load_metric("accuracy")
    predictions = np.argmax(p.predictions, axis=1)
    return metric.compute(predictions=predictions, references=p.label_ids)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Initialize data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Initialize CustomTrainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

# Print sample batch
print("\nSample batch:")
batch = data_collator([train_dataset[i] for i in range(4)])
for k, v in batch.items():
    print(f"{k}: shape {v.shape}, dtype {v.dtype}")

# Train the model
try:
    trainer.train()
except Exception as e:
    print(f"An error occurred during training: {str(e)}")
    print("Input shape:", batch['input_ids'].shape)
    raise

# Evaluate the model
results = trainer.evaluate()
print("Evaluation results:", results)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Charangan/MedBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Dataset info:
DatasetDict({
    train: Dataset({
        features: ['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type', 'admit_provider_id', 'admission_location', 'discharge_location', 'insurance', 'language', 'marital_status', 'race', 'edregtime', 'edouttime', 'hospital_expire_flag'],
        num_rows: 275
    })
})

Dataset columns: ['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type', 'admit_provider_id', 'admission_location', 'discharge_location', 'insurance', 'language', 'marital_status', 'race', 'edregtime', 'edouttime', 'hospital_expire_flag']

Sample from tokenized dataset:
{'input_ids': [101, 158, 2069, 16523, 15681, 157, 9664, 12412, 2271, 9637, 143, 21564, 2107, 145, 9025, 23203, 9159, 2162, 17447, 17656, 17516, 2137, 151, 19556, 13882, 11780, 6820, 19747, 2162, 12150, 3663, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.000307,1.000000


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.000307,1.000000
2,No log,0.000152,1.000000
3,No log,0.000129,1.000000


Evaluation results: {'eval_loss': 0.000129148960695602, 'eval_accuracy': 1.0, 'eval_runtime': 85.7589, 'eval_samples_per_second': 1.166, 'eval_steps_per_second': 0.292, 'epoch': 3.0}


In [25]:
# Define a mapping from numeric class labels to descriptive names
class_labels = {
    0: "LOW",
    1: "MEDIUM",
    2: "HIGH"
}

# Example sample input data formatted as a list of dictionaries
sample_inputs = [
    {
        "admission_type": "Emergency",
        "admission_location": "ICU",
        "discharge_location": "Home",
        "age": 70,
        "gender": "female",
        "ethnicity": "Caucasian",
        "marital_status": "single",
        "employment_status": "unemployed",
        "chronic_conditions": ["asthma", "heart disease"],
        "previous_admissions": 5,
        "length_of_stay": 7,  # in days
        "primary_diagnosis": "chronic obstructive pulmonary disease",
        "medications": ["albuterol", "lisinopril"],
        "discharge_disposition": "home",
    }
]

# Tokenize the sample inputs
def tokenize_sample(inputs):
    # Construct text from available fields
    texts = [f"{item['admission_type']} {item['admission_location']} {item['discharge_location']} {item.get('age', '')} {item.get('gender', '')} {item.get('ethnicity', '')} {item.get('marital_status', '')} {item.get('employment_status', '')} {', '.join(item.get('chronic_conditions', []))} {item.get('previous_admissions', '')} {item.get('length_of_stay', '')} {item.get('primary_diagnosis', '')} {', '.join(item.get('medications', []))} {item.get('discharge_disposition', '')}" for item in inputs]

    # Tokenize
    tokenized_samples = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=275,
        return_tensors='pt'
    )
    return tokenized_samples

# Tokenize the sample inputs
tokenized_samples = tokenize_sample(sample_inputs)

# Move tensors to the same device as the model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Make predictions
model.eval()
with torch.no_grad():
    inputs = {k: v.to(device) for k, v in tokenized_samples.items()}
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1)

# Print predictions with descriptive class names
print("\nPredictions for sample inputs:")
for i, sample in enumerate(sample_inputs):
    predicted_class_index = predictions[i].item()
    predicted_class_name = class_labels.get(predicted_class_index, "Unknown")
    print(f"Sample {i + 1}:")
    print(f"  Input: {sample}")
    print(f"  Predicted class: {predicted_class_name}")



Predictions for sample inputs:
Sample 1:
  Input: {'admission_type': 'Emergency', 'admission_location': 'ICU', 'discharge_location': 'Home', 'age': 70, 'gender': 'female', 'ethnicity': 'Caucasian', 'marital_status': 'single', 'employment_status': 'unemployed', 'chronic_conditions': ['asthma', 'heart disease'], 'previous_admissions': 5, 'length_of_stay': 7, 'primary_diagnosis': 'chronic obstructive pulmonary disease', 'medications': ['albuterol', 'lisinopril'], 'discharge_disposition': 'home'}
  Predicted class: MEDIUM
